In [1]:
import os
import pandas as pd
from pathlib import Path
import yt_dlp
import numpy as np
from tqdm import tqdm

### 동영상 수집 통합본.csv에 영상 업로드 날짜 추가

In [ ]:
df = pd.read_csv("data_kr/video/동영상 수집 통합본.csv")

upload_dt = []
for url in tqdm(df["url"], total=len(df["url"])):
    if pd.isna(url) or url == '':
        upload_dt.append(None)
        continue

    try:
        ydl_opts = {'quiet': True, 'skip_download': True}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            upload_dt.append(info.get("upload_date"))
    except Exception as e:
        upload_dt.append(None)
        print(url)

df["upload_date"] = upload_dt
df["upload_date"] = df["upload_date"].dropna().astype(int).astype(str)
df["upload_date"] = pd.to_datetime(df["upload_date"], format='%Y%m%d').dt.strftime('%Y-%m-%d')
df.to_csv("data_kr/video/동영상 수집 통합본.csv", index=False)

### 업로드 날짜로부터 공시일까지 가격추이 구하기

In [24]:
sectors = ["산업재", "정보기술"]
df = pd.DataFrame()

### LLM 예측 파일 concat ###
for file in Path("preprocessed_data/llm/predict").rglob("*"):
    if file.is_file():
        df_ = pd.read_csv(file)
        df = pd.concat([df, df_], axis=0)
 
### LLM 예측에 실제 등락 라벨 추가 ###
df["code"] = df["code"].astype(str).str.zfill(6)

price_upload = [] # 업로드 당일 종가
price_closure = [] # 공시 당일 종가
for row in tqdm(df.itertuples(), total=len(df)):
    code = str(row.code).zfill(6)
    df_price = pd.read_csv(f"data_kr/price/{code}.csv")
    ### 업로드 날짜 직전 종가
    price_upload.append(df_price.loc[df_price["날짜"] < row.upload_date, "종가"].iloc[-1] if not pd.isna(row.upload_date) else None)
    ### 공시일 날짜 직후 종가
    price_closure.append(df_price.loc[df_price["날짜"] >= row.disclosure_date, "종가"].iloc[0] if not pd.isna(row.disclosure_date) else None)

df["price_upload"] = price_upload
df["price_closure"] = price_closure
label = df["price_upload"] < df["price_closure"] # (업로드 당일 종가)보다 (공시 당일 종가)가 올랐는가?
df["label"] = ["up" if l == True else "down" for l in label]
df["prediction_match"] = df["prediction"] == df["label"]

df = df[["code", "name", "sector", "year", "quarter", "upload_date", "disclosure_date", "price_upload", "price_closure", "label", "prediction", "prediction_match"]]
df.to_csv("preprocessed_data/llm/predict/업로드_공시_가격비교.csv", index=False)

100%|█████████████████████████████████████████████████████████████████████████████| 2940/2940 [00:10<00:00, 291.96it/s]


### columns=["name", "sector"] 한글 깨짐 수정

In [61]:
df=pd.read_csv("data_kr/video/merged.csv")

df.loc[df["code"] == 660, "name"] = "SK하이닉스"
df.loc[df["code"] == 660, "sector"] = "정보기술"

df.loc[df["code"] == 3550, "name"] = "LG"
df.loc[df["code"] == 3550, "sector"] = "정보기술"

df.loc[df["code"] == 4710, "name"] = "한솔테크닉스"
df.loc[df["code"] == 4710, "sector"] = "정보기술"

df.loc[df["code"] == 5930, "name"] = "삼성전자"
df.loc[df["code"] == 5930, "sector"] = "정보기술"

df.loc[df["code"] == 6400, "name"] = "삼성SDI"
df.loc[df["code"] == 6400, "sector"] = "정보기술"

df = df.sort_values(by=['code', 'disclosure_date'], ascending=[True, True])

df.to_csv("data_kr/video/merged.csv", index=False,  encoding='utf-8')

### 수정된 행들만 별도 파일로 저장

In [84]:
df_o = pd.read_csv("data_kr/video/동영상 수집 통합본_origin.csv")
df_m = pd.read_csv("data_kr/video/merged.csv")

df_o = df_o.sort_values(by=['code', 'disclosure_date'], ascending=[True, True]).reset_index(drop=True)
df_m = df_m.sort_values(by=['code', 'disclosure_date'], ascending=[True, True]).reset_index(drop=True)

df_diff = df_m[df_o["url"] != df_m["url"]]
sum(df_diff["category"] == "video")

16

In [81]:
df_diff.to_csv("data_kr/video/수정된 종목들.csv", index=False, encoding="utf-8")

,year,quarter,disclosure_date,code,name,sector,url,category
0,2016,Q1,2016-05-16,120,CJ대한통운,산업재,https://www5.ajunews.com/view/20160504095741793,article
2,2016,Q3,2016-11-14,120,CJ대한통운,산업재,https://www.klnews.co.kr/news/articleView.html...,article
3,2016,Q4,2017-03-31,120,CJ대한통운,산업재,https://magazine.hankyung.com/business/article...,article
5,2017,Q2,2017-08-14,120,CJ대한통운,산업재,https://www.edaily.co.kr/News/Read?newsId=0276...,article
8,2018,Q1,2018-05-15,120,CJ대한통운,산업재,https://news.nate.com/view/20180511n27428?mid=...,article
...,...,...,...,...,...,...,...,...
968,2021,Q4,2022-03-15,86280,현대글로비스,산업재,https://www.g-enews.com/article/Industry/2022/...,article
972,2022,Q4,2023-03-21,86280,현대글로비스,산업재,https://www.securityfact.co.kr/3684,article
973,2023,Q1,2023-05-15,86280,현대글로비스,산업재,https://www.hankyung.com/article/2023050803751,article
975,2023,Q3,2023-11-14,86280,현대글로비스,산업재,https://www.asiae.co.kr/article/20231107065019...,article


https://www5.ajunews.com/view/20160504095741793
https://www.klnews.co.kr/news/articleView.html?idxno=114432
https://magazine.hankyung.com/business/article/201703270765b
https://www.edaily.co.kr/News/Read?newsId=02765046616025352&mediaCodeNo=257
https://news.nate.com/view/20180511n27428?mid=n0000
https://news.nate.com/view/20180808n14859
https://www.labortoday.co.kr/news/articleView.html?idxno=161418
https://news.nate.com/view/20200329n02205?mid=n0000
https://www.mk.co.kr/news/business/7336629
https://news.nate.com/view/20161111n11422?mid=n0901
https://www5.ajunews.com/view/20170328144008130
https://news.nate.com/view/20170510n38234?mid=n0000
https://www.munhwa.com/article/11046214
https://www.ajunews.com/view/20180511091819814
https://biz.heraldcorp.com/article/1748405
https://www5.ajunews.com/view/20181111135039726
https://www.doosannewsroom.com/?p=38761&cat=8
https://news.nate.com/view/20190514n35766?mid=n0000
https://www.kipost.net/news/articleView.html?idxno=201580
https://news.nat

In [9]:
import shutil

df=pd.read_csv("data_kr/video/동영상 수집 통합본 최신.csv")
for row in df[df["category"] == "article"].itertuples():
    code = str(row.code).zfill(6)
    text_dir = f'data_kr/video/text/{row.sector}/{code}/'
    summary_dir = f'preprocessed_data/llm/summary/{row.sector}/{code}/'
    
    shutil.copy(f'{text_dir}{row.year}-{row.quarter}.txt', summary_dir)